### Load the dataset

In [5]:
from models import ClaimsDataset, Claim

with open("data/claims_dataset_v2_manual.json", "r") as f:
    dataset = ClaimsDataset.model_validate_json(f.read())
print(len(dataset.root))

400


In [3]:
print(dataset.root[0].model_dump_json(indent=2))

{
  "description": "While driving on the highway, my car was involved in a collision. The impact caused shattered glass from the windshield, resulting in my spouse suffering a total and irrecoverable loss of sight in one eye. Is there a provision for compensation specifically for my spouse's injury under the policy limits?",
  "explanation": "",
  "coverage": true,
  "sources": [
    {
      "paragraph": "If: - you or your spouse are injured solely as a result of an accident involving your car; or while travelling in or getting into or out of your car; or - any other person is injured -while travelling in or getting into or out of your car; we will pay you or your legal representatives or if you ask us to, the person injured or their legal representative the compensation specified below: Provided that death or loss occurs within three calendar months and solely and directly as a result of the accident.",
      "file_name": "",
      "page": 21,
      "line": 0
    },
    {
      "parag

### Split Train / Test

In [6]:
import random

# Seed for reproducibility
random.seed(42)

# shuffle the dataset
random.shuffle(dataset.root)
train_size = int(len(dataset.root) * 0.8)
train_dataset: list[Claim] = dataset.root[:train_size]
test_dataset: list[Claim] = dataset.root[train_size:]

len(train_dataset), len(test_dataset)

(320, 80)

### Establish baseline performance Qwen2.5(3B)

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output